In [2]:
import os, sys
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import darts
from darts.utils.statistics import check_seasonality, plot_acf, stationarity_tests
from darts.dataprocessing.transformers.missing_values_filler import MissingValuesFiller
from darts.dataprocessing.transformers.boxcox import BoxCox
from darts.dataprocessing.transformers.diff import Diff
from darts.utils.statistics import plot_hist
from darts.models import LightGBMModel, XGBModel, LinearRegressionModel, TFTModel, NHiTSModel, RNNModel, TFTModel
from darts.metrics import smape, mape, mase, mse, rmse, r2_score, mae
from darts.dataprocessing.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler   
from darts.dataprocessing.transformers.scaler import Scaler

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks import ModelCheckpoint
import torch

from utils import *
import wandb
wandb.login()


import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)



In [3]:

def predict_testset(model, ts, ts_covs):

    print('Predicting test set...')
    historics = model.historical_forecasts(ts, 
                                        future_covariates= ts_covs,
                                        start=ts_test_piped.get_index_at_point(n_lags),
                                        verbose=False,
                                        stride=3, 
                                        forecast_horizon=n_ahead, 
                                        retrain=False, 
                                        last_points_only=False, # leave this as False unless you want the output to be one series, the rest will not work with this however
                                        )
    
    
    ts_predictions = ts_list_concat(historics) # concatenating the batches into a single time series for plot 1, this keeps the n_ahead
    ts_predictions_inverse = pipeline.inverse_transform(ts_predictions) # inverse transform the predictions, we need the original values for the evaluation
    return ts_predictions_inverse.pd_series().to_frame('prediction')



def ts_list_concat(ts_list):
    '''This function concatenates a list of time series into one time series, depending on what the stride was in the eval pipeline'''
    ts = ts_list[0]
    for i in range(1, len(ts_list)-1):
        previous_end = ts.end_time()
        ts = ts[:-1].append(ts_list[i][previous_end:])
    return ts


In [4]:
# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')

Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\WattCast


In [5]:
# run parameters

config_dataset = {
    'spatial_scale': '1_county',
    'temp_resolution': 60,
    'location': 'New_York',
}

config_modeldesign = {'boxcox': True,
                    'horizon_in_hours': 24, # in hours
                    'lookback_in_hours': 24, # in hours
                    'liklihood': None,
                    'weather': True,
                    'holiday': True,
                    'datetime_encodings': True,
                    }
                   
config_encoders =  {
                    "cyclic": {"future": ["hour"]}, 
                    "position": {"future": ["relative",]},
                    "datetime_attribute": {"future": ["dayofweek", "week"]},
                    'position': {'past': ['relative'], 'future': ['relative']},
            }



In [6]:
# calculate derived parameters
datetime_encoders = config_encoders if config_modeldesign['datetime_encodings'] else None
timesteps_per_hour = int(60 / config_dataset['temp_resolution'])
n_lags = config_modeldesign['lookback_in_hours'] * timesteps_per_hour
n_ahead = config_modeldesign['horizon_in_hours'] * timesteps_per_hour
list_metrics = [smape, mape, rmse, r2_score, mae] # evaluation metrics
eval_stride = n_ahead # evaluation stride, how often to evaluate the model, in this case we evaluate every n_ahead steps

# Loading Data
df_train = pd.read_hdf(os.path.join(dir_path, f'{config_dataset["spatial_scale"]}.h5'), key=f'{config_dataset["location"]}/{config_dataset["temp_resolution"]}min/train_target')
df_val = pd.read_hdf(os.path.join(dir_path, f'{config_dataset["spatial_scale"]}.h5'), key=f'{config_dataset["location"]}/{config_dataset["temp_resolution"]}min/val_target')
df_test = pd.read_hdf(os.path.join(dir_path, f'{config_dataset["spatial_scale"]}.h5'), key=f'{config_dataset["location"]}/{config_dataset["temp_resolution"]}min/test_target')
df_cov_train = pd.read_hdf(os.path.join(dir_path, f'{config_dataset["spatial_scale"]}.h5'), key=f'{config_dataset["location"]}/{config_dataset["temp_resolution"]}min/train_cov')
df_cov_val = pd.read_hdf(os.path.join(dir_path, f'{config_dataset["spatial_scale"]}.h5'), key=f'{config_dataset["location"]}/{config_dataset["temp_resolution"]}min/val_cov')
df_cov_test = pd.read_hdf(os.path.join(dir_path, f'{config_dataset["spatial_scale"]}.h5'), key=f'{config_dataset["location"]}/{config_dataset["temp_resolution"]}min/test_cov')

In [7]:
# into darts format
ts_train = darts.TimeSeries.from_dataframe(df_train)
ts_val = darts.TimeSeries.from_dataframe(df_val)
ts_test = darts.TimeSeries.from_dataframe(df_test)

# Covariates
ts_cov_train = darts.TimeSeries.from_dataframe(df_cov_train)
ts_cov_val = darts.TimeSeries.from_dataframe(df_cov_val)[:1000]
ts_cov_test = darts.TimeSeries.from_dataframe(df_cov_test)[:1000]

# make sure the indices are the same, TODO: move this to the data_prep.ipynb
ts_train, ts_cov_train = make_index_same(ts_train, ts_cov_train)
ts_val, ts_cov_val = make_index_same(ts_val, ts_cov_val)
ts_test, ts_cov_test = make_index_same(ts_test, ts_cov_test)

In [8]:
# Load pipeline
pipeline = Pipeline(
                    [
                    BoxCox() if config_modeldesign['boxcox'] else None,
                    Scaler(MinMaxScaler()),
                    ]
                     )

ts_train_piped = pipeline.fit_transform(ts_train)
ts_val_piped = pipeline.transform(ts_val)
ts_test_piped = pipeline.transform(ts_test)

# Weather Pipeline
pipeline_weather = Pipeline([Scaler(RobustScaler())])
ts_train_weather_piped = pipeline_weather.fit_transform(ts_cov_train)
ts_val_weather_piped = pipeline_weather.transform(ts_cov_val)
ts_test_weather_piped = pipeline_weather.transform(ts_cov_test)

In [9]:
trg_train_inversed = pipeline.inverse_transform(ts_train_piped, partial=True) # inverse transform the target, we need the original values for the evaluation
trg_val_inversed = pipeline.inverse_transform(ts_val_piped, partial=True) # inverse transform the target, we need the original values for the evaluation
trg_test_inversed = pipeline.inverse_transform(ts_test_piped, partial=True) # inverse transform the target, we need the original values for the evaluation

## Hyperparameter Tuning with wandb sweep

### XGBoost

### LightGBM

### GRU Model

In [11]:
def train_gru():

    wandb.init()
    config = wandb.config

    optimizer_kwargs = {}

    optimizer_kwargs['lr'] = config.lr
    
    n_lags = config.lookback_in_hours * timesteps_per_hour

    pl_trainer_kwargs = {
        'max_epochs': 50,
        'accelerator': 'gpu',
        'devices': [0],
        'callbacks': [EarlyStopping(monitor='val_loss', patience=5, mode='min')],
        'logger': WandbLogger(log_model='all'),
    }

    schedule_kwargs = {
        'patience': 2,
        'factor': 0.5,
        'min_lr': 1e-5,
        'verbose': True
        }

    model = RNNModel(  
                    model = 'GRU',
                    input_chunk_length=n_lags,
                    output_chunk_length=n_ahead,
                    hidden_dim=config.hidden_dim,
                    n_rnn_layers=config.n_rnn_layers,
                    batch_size=config.batch_size,
                    dropout=config.dropout,
                    add_encoders=datetime_encoders,
                    likelihood=None,
                    pl_trainer_kwargs=pl_trainer_kwargs,
                    optimizer_kwargs=optimizer_kwargs,
                    lr_scheduler_cls=ReduceLROnPlateau,
                    lr_scheduler_kwargs=schedule_kwargs,
                    random_state=42,
                )

    model.fit(ts_train_piped, future_covariates = ts_cov_train, val_series=ts_val_piped, val_future_covariates=ts_cov_val, verbose=True)

    predictions = predict_testset(model, ts_test_piped[:200], ts_cov_test[:200]) # visualize only the first 200 time steps

    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'predictions': fig})


    wandb.finish()


config_sweep_gru = {
    'name': 'GRU sweep' + config_dataset['spatial_scale'] + '_' + config_dataset['location'] + '_' + str(config_dataset['temp_resolution']),
    'method': 'bayes', #grid, random
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'lr': {
            'values': [5e-3, 1e-3, 3e-4]
},
        'hidden_dim': {
            'values': [64, 512, 1024]
},
        'n_rnn_layers': {
            'values': [1, 2, 3]
},
        'dropout': {
            'values': [0.1, 0.2, 0.3]
},
        'batch_size': {
            'values': [32, 64, 128]
},
        'lookback_in_hours': {
            'values': [24]
},
}
}


sweep_id = wandb.sweep(config_sweep_gru, project="WattCast")
wandb.agent(sweep_id, train_gru, count=20)

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'hidden_dim' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dropout' was locked by 'sweep' (ignored update).


Predicting test set...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'hidden_dim' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dropout' was locked by 'sweep' (ignored update).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'hidden_dim' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'dropout' was locked by 'sweep' (ignored update).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'hidden_dim' was locked by 'sweep' (ignored update).
wandb: WARNING Config it

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train_loss,█▇▄▄▅▂▆▁▃▆▃▂▂▄▅▄▅▂▇▂▃▄█▄▃▃▂▂▃▁▃▄▄▇▃▆▂▁▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
val_loss,▅▄█▂▁▄▄▁▄▂
epoch,9
train_loss,0.03525
trainer/global_step,2749
val_loss,0.01942


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


### Transformer Model

### D-Linear - https://arxiv.org/abs/2205.13504

In [12]:
from darts.models import DLinearModel, NLinearModel

def train_dlinear():

    wandb.init()
    config = wandb.config

    optimizer_kwargs = {}

    optimizer_kwargs['lr'] = config.lr
    
    n_lags = config.lookback_in_hours * timesteps_per_hour

    pl_trainer_kwargs = {
        'max_epochs': 50,
        'accelerator': 'gpu',
        'devices': [0],
        'callbacks': [EarlyStopping(monitor='val_loss', patience=5, mode='min')],
        'logger': WandbLogger(log_model='all'),
    }

    schedule_kwargs = {
        'patience': 2,
        'mode': 'min',
        'threshold': 0.01,
        'factor': 0.5,
        'min_lr': 1e-5,
        'verbose': True
        }

    model = DLinearModel( 
                    kernel_size=config.kernel_size, 
                    const_init=config.const_init,
                    input_chunk_length=n_lags,
                    output_chunk_length=n_ahead,
                    batch_size=config.batch_size,
                    add_encoders=datetime_encoders,
                    likelihood=None,
                    pl_trainer_kwargs=pl_trainer_kwargs,
                    optimizer_kwargs=optimizer_kwargs,
                    lr_scheduler_cls=ReduceLROnPlateau,
                    lr_scheduler_kwargs=schedule_kwargs,
                    random_state=42,
                )

    model.fit(ts_train_piped, future_covariates = ts_cov_train, val_series=ts_val_piped, val_future_covariates=ts_cov_val, verbose=True)

    predictions = predict_testset(model, ts_test_piped[:200], ts_cov_test[:200]) # visualize only the first 200 time steps

    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'predictions': fig})


    wandb.finish()


config_sweep_dlinear = {
    'name': 'D-Linear sweep' + config_dataset['spatial_scale'] + '_' + config_dataset['location'] + '_' + str(config_dataset['temp_resolution']),
    'method': 'bayes', #grid, random
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'lr': {
            'values': [5e-3, 1e-3, 3e-4]
},
        'kernel_size': {
            'values': [15, 25, 51, 101]
},
        'batch_size': {
            'values': [32, 64, 128]
},

'const_init': {
            'values': [True, False]
},
        'lookback_in_hours': {
            'values': [24]
},
}
}


sweep_id = wandb.sweep(config_sweep_dlinear, project="WattCast")
wandb.agent(sweep_id, train_dlinear, count=20)

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'kernel_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'const_init' was locked by 'sweep' (ignored update).


Predicting test set...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'kernel_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'const_init' was locked by 'sweep' (ignored update).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'kernel_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'const_init' was locked by 'sweep' (ignored update).
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING Config item 'kernel_size' was locked by 'sweep' (ignored update).
wandb: WARNING 

Exception in thread Thread-33 (_run_job):
Traceback (most recent call last):
  File "c:\Users\nik\miniconda3\envs\gpu2\lib\site-packages\wandb\sdk\wandb_run.py", line 2092, in _atexit_cleanup
    self._on_finish()
  File "c:\Users\nik\miniconda3\envs\gpu2\lib\site-packages\wandb\sdk\wandb_run.py", line 2325, in _on_finish
    _ = exit_handle.wait(timeout=-1, on_progress=self._on_progress_exit)
  File "c:\Users\nik\miniconda3\envs\gpu2\lib\site-packages\wandb\sdk\lib\mailbox.py", line 298, in wait
    on_probe(probe_handle)
  File "c:\Users\nik\miniconda3\envs\gpu2\lib\site-packages\wandb\sdk\wandb_run.py", line 2290, in _on_probe_exit
    result = handle.wait(timeout=0)
  File "c:\Users\nik\miniconda3\envs\gpu2\lib\site-packages\wandb\sdk\lib\mailbox.py", line 281, in wait
    raise MailboxError("transport failed")
wandb.sdk.lib.mailbox.MailboxError: transport failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\User